# Chapter 4: Exploratory Data Analysis

---

## Objective
Describe briefly what this chapter covers.

## TODO: Code Examples
Paste and run code examples from the PDF here.

## Results / Plots
Add outputs, figures, or screenshots here.

## Insights
Write 2–3 bullet points of what you learned.